## MNIST Tensorflow2.0 실습

In [205]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 데이터 불러오기

In [366]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()

In [367]:
(X, y) = datasets.load_digits(return_X_y=True)
X = X.astype(np.float32)
y = y.astype(np.int32)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2, random_state=1)

In [368]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1437, 64)
(360, 64)
(1437,)
(360,)


### Hidden Layer 숫자를 지정

In [209]:
n_inputs = 64
n_hidden_1 = 25
n_hidden_2 = 25
n_outputs = 10

### 신경망 클래스 빌드

In [210]:
class NNLayer(object):
    def __init__(self, n_inputs, n_neurons, activation=None):
        init = tf.random.normal((n_inputs, n_neurons),
                                stddev=2 / np.sqrt(n_inputs))
        self.W = tf.Variable(init, name='kernel')
        self.b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        self.activation = activation
        
    def __call__(self, X):
        Z = tf.matmul(X, self.W) + self.b
        if self.activation is not None:
            return self.activation(Z)
        else:
            return Z
            
    def trainable_variables(self):
        return [self.W, self.b]
        

In [211]:
class Model(object):
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, X):
        for layer in self.layers:
            X = layer(X)
        return X
    
    def trainable_variables(self):
        variables = []
        for layer in self.layers:
            variables.extend(layer.trainable_variables())
        return variables

In [212]:
# Loss Function 
loss_function = tf.nn.sparse_softmax_cross_entropy_with_logits
#loss_function = tf.nn.softmax_cross_entropy_with_logits

### Hidden Layer 만들기

In [213]:
model = Model([
    NNLayer(n_inputs, n_hidden_1, activation=tf.nn.relu),
    NNLayer(n_hidden_1, n_hidden_2, activation=tf.nn.relu),
    NNLayer(n_hidden_2, n_outputs)
])

In [214]:
def loss(model, X, y_true):
    y_pred = model(X)
    return tf.reduce_mean(tf.dtypes.cast(loss_function(labels=y_true, logits=y_pred),
                                         tf.float32, name='loss'))

In [215]:
l = loss(model, X_train, y_train)
print('Testing loss:', l)

Testing loss: tf.Tensor(42.716923, shape=(), dtype=float32)


In [216]:
# GD Backpropagation
def grad(model, X, y_true):
    with tf.GradientTape() as tape:
        loss_value = loss(model, X, y_true)
    return loss_value, tape.gradient(loss_value, model.trainable_variables())

In [217]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [218]:
loss_value, grads = grad(model, X_train, y_train)

print('Step: {}, Initial Loss:{}'.format(optimizer.iterations.numpy(),
                                        loss_value.numpy()))

Step: 0, Initial Loss:42.716922760009766


In [219]:
optimizer.apply_gradients(zip(grads, model.trainable_variables()))

print('Step: {}, Initial Loss:{}'.format(optimizer.iterations.numpy(),
                                        loss(model, X_train, y_train).numpy()))

Step: 1, Initial Loss:26.0883846282959


### Epochs로 학습하기

In [220]:
train_loss_results=[]
train_accuracy_results=[]

epochs=100

for epoch in range(epochs):
    
    loss_value, grads = grad(model, X_train, y_train)
    optimizer.apply_gradients(zip(grads, model.trainable_variables()))
    
    y_pred = model(X_train)
    correct = tf.equal(tf.math.argmax(y_pred, axis=1), y_train)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    train_loss_results.append(loss_value.numpy())
    train_accuracy_results.append(accuracy)
    
    if epoch % 5 == 0:
        print('Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}'.format(epoch,
                                                                   loss_value.numpy(),
                                                                   accuracy))
    
    

Epoch 000: Loss: 26.088, Accuracy: 18.163%
Epoch 005: Loss: 7.199, Accuracy: 23.382%
Epoch 010: Loss: 4.242, Accuracy: 29.436%
Epoch 015: Loss: 2.396, Accuracy: 41.475%
Epoch 020: Loss: 1.983, Accuracy: 47.112%
Epoch 025: Loss: 1.600, Accuracy: 53.166%
Epoch 030: Loss: 1.402, Accuracy: 57.550%
Epoch 035: Loss: 1.219, Accuracy: 63.187%
Epoch 040: Loss: 1.078, Accuracy: 66.667%
Epoch 045: Loss: 0.949, Accuracy: 71.190%
Epoch 050: Loss: 0.831, Accuracy: 75.922%
Epoch 055: Loss: 0.736, Accuracy: 78.288%
Epoch 060: Loss: 0.655, Accuracy: 80.932%
Epoch 065: Loss: 0.591, Accuracy: 82.324%
Epoch 070: Loss: 0.538, Accuracy: 83.786%
Epoch 075: Loss: 0.496, Accuracy: 84.830%
Epoch 080: Loss: 0.459, Accuracy: 86.013%
Epoch 085: Loss: 0.426, Accuracy: 86.848%
Epoch 090: Loss: 0.394, Accuracy: 87.404%
Epoch 095: Loss: 0.367, Accuracy: 88.518%


## MNIST Tensorflow2 실습 2

In [391]:
import tensorflow as tf

In [392]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = X_train / 255.0

In [393]:
train_tf = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_tf = train_tf.shuffle(1000).batch(32)

In [394]:
X_train.shape

(60000, 28, 28)

In [395]:
X_test.shape

(10000, 28, 28)

In [396]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

def my_model():
    inputs = Input(shape=(28, 28))
    
    x = Flatten()(inputs)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(10, activation='softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    
    return model

model = my_model()

In [397]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

loss_object = SparseCategoricalCrossentropy()
optimizer = Adam()

In [398]:
from tensorflow.keras.metrics import Mean
from tensorflow.keras.metrics import SparseCategoricalAccuracy

train_loss = Mean(name='train_loss')
train_accuracy = SparseCategoricalAccuracy(name='train_accuracy')

In [399]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape as tape:
        outputs = model(images, training=True)
        loss = loss_object(labels, outputs)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, outputs)
        

In [400]:
Epochs = 2

for i in range(Epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for images, labels in train_tf:
        train_step(images, labels)
        
    print('Epoch: {}, train_loss: {}, train_acc: {}'.format(
    i+1,
    train_loss.result(),
    train_accuracy.result()*100))

AttributeError: in converted code:

    <ipython-input-342-462a443fc87e>:3 train_step  *
        with tf.GradientTape as tape:

    AttributeError: __enter__
